# Código para identificar tópicos presentes en el corpus noticioso de 'El Siglo de Torreón'

## Importes Necesarios

In [ ]:
#Numpy and Pandas (necesarios para Spacy)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib inline

#gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

#NLTK
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

#spacy y stanza
import spacy
nlp = spacy.load("es_core_news_sm")

#mysql
import mysql.connector as sql

#extras
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Definiciones

In [ ]:
#Definición de constantes para conexión MySQL
host_c="localhost"
port_c=3306
user_c="root"
passwd_c="Password1234!"
db_c="siglodb_2002-2022"

conn = sql.connect(host=host_c, port=port_c, user=user_c, passwd=passwd_c, db=db_c)
cursor = conn.cursor(buffered=True)      

exceptions = {"biden", "otan", "florida"}

#Función para preprocesamiento de textos utilizando Gensim
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True, min_len=4)
        final.append(new)
    return (final)

#Función para remover palabras de uso común
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

#Funciones para creación de bigramas y trigramas
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

#Función para lematización de los textos
def lemmatization(texts):
    texts_out = []
    for text in tqdm(texts):
        doc = nlp(" ".join(text)) 
        lemmatized_text = []
        for token in doc:
            if token.text.lower() in exceptions:
                lemmatized_text.append(token.text)  # Keep the original word
            else:
                lemmatized_text.append(token.lemma_)
        texts_out.append(lemmatized_text)
    return texts_out

#Función para evaluar puntaje de coherencia de los tópicos resultantes
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=4):
    coherence_values = []
    model_list = []
    for num_topics in tqdm(range(start, limit, step)):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, alpha='auto', eta=.5, passes=20, chunksize=100, update_every=1, random_state=100)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        
    return model_list, coherence_values

#Función para ordenar los tópicos
def Sort(sub_li):
     sub_li.sort(key = lambda x: x[1])
     sub_li.reverse()
     return (sub_li)

#Función para insertar los tópicos resultantes (Clave de tópico, identificador, palabras que componen el tópico, peso de cada palabra dentro del tópico y distribución marginal de cada palabra en el tópico) en la base de datos MySQL
def insert_into_mysql(row):
    cursor = conn.cursor(buffered=True)
    query = "INSERT INTO siglodb_topicos (topico_clave, topic_num, palabra, peso, distribucion_marginal) VALUES (%s, %s, %s, %s, %s)"
    values = (row['topico_clave'], row['topic_num'], row['palabra'], row['peso'], row['distribucion_marginal'])
    cursor.execute(query, values)
    conn.commit()
    cursor.close()

#Función para registrar el tópico de cada una de las noticias en la base de datos general
def update_row_in_database(row):
    cursor = conn.cursor(buffered=True)
    cursor.execute("UPDATE siglodb2022 SET topico1 = %s, topico1_peso = %s, \
                    topico2 = %s, topico2_peso = %s \
                    WHERE id = %s",
                   (row['topico1'], row['topico1_peso'], row['topico2'], row['topico2_peso'], row['id']))
    conn.commit()
    cursor.close()

#Función para extraer tuplas y poder registrar los tópicos de la función anterior
def extract_tuple(row):
    if row is None:
        return pd.Series([None, None])
    return pd.Series(row)

## Inicio del análisis

In [ ]:
#Definición de la fecha de inicio de la iteración y la sección a analizar
Mes = 1
Año = 2022
Seccion = "Regionales"

In [ ]:
for i in range(13): 
    if Mes == 13:
        break  
    
    #Conectar a la base de datos en MySQL local
    distintivo = f"{Año}_{Mes}_{Seccion}"
    conn = sql.connect(host=host_c, port=port_c, user=user_c, passwd=passwd_c, db=db_c)
    cursor = conn.cursor(buffered=True)      
    query = "SELECT primer_parrafo FROM siglodb2022 WHERE MONTH(fecha) = %s AND YEAR(fecha) = %s AND seccion = %s;"
    cursor.execute(query, (Mes, Año, Seccion))
    data = cursor.fetchall()
    cursor.close()
    titulares = [row[0] for row in data]
    
    #Tokeniza
    data_words = gen_words(titulares)
    
    #Remueve palabras de uso común
    stop_words = stopwords.words('spanish')
    stop_words.extend(['cada', 'persona', 'mil', 'tras', 'ademas', 'pues', 'habia', 'debido', 'aunque', 'también', 'tambien', 'lunes', 'martes', 'miércoles', 'jueves', 'sábado', 'domingo', 'miercoles', 'dijo', 'según', 'segun', 'estan', 'están', 'puede', 'debe', 'años', 'año', 'anos', 'ano', 'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre'])
    data_words_nostops = remove_stopwords(data_words)
    
    #Genera bigramas y trigramas
    bigram = gensim.models.Phrases(data_words_nostops, min_count=20, threshold=200)
    trigram = gensim.models.Phrases(bigram[data_words_nostops], min_count=20, threshold=200)  
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    data_words_bigrams = make_bigrams(data_words_nostops)
    data_words_trigrams = make_trigrams(data_words_bigrams)
    
    #Lematiza
    data_lemmatized = lemmatization(data_words_trigrams)
    
    #Genera lista y remueve palabras con alta frecuencia
    id2word = corpora.Dictionary(data_lemmatized)
    texts = data_lemmatized
    corpus = [id2word.doc2bow(text) for text in texts]
    tfidf = TfidfModel(corpus, id2word=id2word)
    low_value = 0.03
    words =[]
    words_missing_in_tfidf =[]
    for i in range(0, len(corpus)):
        bow = corpus[i]
        low_value_words = []
        tfidf_ids = [id for id, value in tfidf[bow]]
        bow_ids = [id for id, value in bow]
        low_value_words = [id for id, value in tfidf[bow] if value < low_value]
        drops = low_value_words+words_missing_in_tfidf
        for item in drops:
            words.append(id2word[item])
        words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
        new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
        corpus[i] = new_bow
    
    #Itera por diferentes modelos según número de tópicos y genera puntajes de coherencia
    model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=2, limit=60, step=4)

    #Genera una gráfica con los puntajes de coherencia y número de tópicos. Guarda la gráfica a disco duro. 
    limit=60
    start=2
    step=4

    x = range(start, limit, step)

    plt.plot(x, coherence_values, label =f"{Mes}")
    plt.xlabel("Num Tópicos")
    plt.ylabel("Coherence score")
    plt.title(f"{Seccion} - {Año}")
    plt.legend(loc='best')
    plt.savefig(f"results/figures/{distintivo}.pdf")
    

    best_model_index = coherence_values.index(max(coherence_values))
    max_score = max(coherence_values)
    best_model = model_list[best_model_index]
    best_num_topics = best_model.num_topics

    # Obtiene los tópicos del modelo con mejor puntaje de coherencia
    topics = best_model.show_topics(formatted=False, num_words=30)

     # Itera por cada uno de los tópicos del mejor modelo y genera la distribución marginal. Si el tópico seleccionado está presente en más del 5% de las noticias, lo selecciona para análisis. 
    selected_topics = []
    for topic_id, topic in topics:     
        marginal_distribution = sum([prob for _, prob in topic])
        if marginal_distribution > 0.005:
            selected_topics.append((topic_id, topic, marginal_distribution))

    # Genera el número de tópicos seleccionados (aquellos con una distribución marginal mayor a 5%)
    num_selected_topics = len(selected_topics)

    #Selecciona el mejor modelo y registra los valores en siglodb_topicos_registro, donde se guarda un registro de cada uno de los modelos, número de tópicos resultantes y número de tópicos seleccionados
    query = "INSERT INTO siglodb_topicos_registro (modelo_clave, coherence_score, num_topicos, num_topicos_seleccionados) VALUES (%s, %s, %s, %s);"
    conn = sql.connect(host=host_c, port=port_c, user=user_c, passwd=passwd_c, db=db_c)
    cursor = conn.cursor(buffered=True)      
    cursor.execute(query, (distintivo, max_score, best_num_topics, num_selected_topics))
    cursor.close()
    conn.commit()
    
    # #Genera la lista de palabras por tópico
    lda_data = []
    for topic_num, words, marginal_distribution in selected_topics:
        for word, weight in words:
            lda_data.append([f"{distintivo}", topic_num, word, weight, marginal_distribution])

    topics_df = pd.DataFrame(lda_data, columns=['topico_clave', 'topic_num', 'palabra', 'peso', 'distribucion_marginal'])
    topics_df['topic_num'] = topics_df['topic_num'].astype(str)
    topics_df['topico_clave'] = topics_df['topico_clave'].str.cat(topics_df['topic_num'], sep='_')
    
    conn = sql.connect(host=host_c, port=port_c, user=user_c, passwd=passwd_c, db=db_c)
    topics_df.apply(insert_into_mysql, axis=1)
    conn.close()
    
    #Guarda el modelo en disco duro
    foldername = f"results/models/{distintivo}"
    os.makedirs(foldername, exist_ok=True)
    best_model.save(f"{foldername}/{distintivo}.model")

    #Genera una visualización de los tópicos del modelo seleccionado y la guarda a disco duro
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(best_model, corpus, id2word, mds="mmds", R=30)
    pyLDAvis.save_html(vis, f"results/visuals/{distintivo}.html")

    #Ordena la lista de tópicos y palabras
    topicos_ordenados = []
    
    for doc in corpus:
        topics = best_model[doc]
        topics_sorted = Sort(topics)
        topicos_ordenados.append(topics_sorted)
    
    # Trunca a dos tópicos por noticia
    topicos_truncos = []
    
    for inner_list in topicos_ordenados:
        truncated_inner_list = inner_list[:2]
        topicos_truncos.append(truncated_inner_list)

    #Genera un DataFrame con todos los tópicos
    columns = ['Tópico 1', 'Tópico2']
    df= pd.DataFrame(topicos_truncos, columns=columns)
    num_rows = len(df)
    print(num_rows)
    
    df['topico_clave'] = [distintivo] * num_rows
    df[['topico1', 'topico1_peso']] = df['Tópico 1'].apply(extract_tuple)
    df[['topico2', 'topico2_peso']] = df['Tópico 2'].apply(extract_tuple)
    df.drop(['Tópico 1'], axis=1, inplace=True)
    
    df['topico1'] = df['topico1'].fillna(-1000)
    df['topico1'] = df['topico1'].astype(int)
    df['topico2'] = df['topico2'].fillna(-1000)
    df['topico2'] = df['topico2'].astype(int)
    
    df['topico1'] = df['topico1'].astype(str)
    df['topico1'] = df['topico_clave'].str.cat(df['topico1'], sep='_')
    df['topico2'] = df['topico2'].astype(str)
    df['topico2'] = df['topico_clave'].str.cat(df['topico2'], sep='_')
    df.drop(['topico_clave'], axis=1, inplace=True)
    
    null_value = f"{distintivo}_-1000"

    a_revision = ['topico1', 'topico2']
    for col in a_revision:
        df.loc[df[col].str.contains(null_value, case=False), col] = None

    #Inserta nuevamente los id's de las noticias en el dataframe y actualiza la información mediante MySQL en la base de datos siglodb2022
    query = "SELECT id FROM siglodb2022 WHERE MONTH(fecha) = %s AND YEAR(fecha) = %s AND seccion = %s;"
    conn = sql.connect(host=host_c, port=port_c, user=user_c, passwd=passwd_c, db=db_c)
    param_values = (Mes, Año, Seccion)
    df_from_sql = pd.read_sql(query, conn, params=param_values)
    df['id'] = df_from_sql['id']

    for index, row in tqdm(df.iterrows()):
        cursor = conn.cursor(buffered=True)
        cursor.execute("SELECT * FROM siglodb2022 WHERE id = %s", (row['id'],))
        result = cursor.fetchone()
        cursor.close()
      
        if result:
            row.replace({pd.NA: None, np.nan: None}, inplace=True)
            update_row_in_database(row)
    
    conn.close()

    #Incrementa la variable "Mes" para iterar por los diferentes meses del año
    Mes += 1 

print("Año completo")